# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 16 2023 10:55AM,255092,22,1162023,FUJIFILM Diosynth Biotechnologies Texas LLC,Released
1,Jan 16 2023 10:54AM,255091,10,PRF-46696,Bio-PRF,Released
2,Jan 16 2023 10:54AM,255091,10,PRF-46705,Bio-PRF,Released
3,Jan 16 2023 10:54AM,255091,10,PRF-46709,Bio-PRF,Released
4,Jan 16 2023 10:54AM,255091,10,PRF-46713,Bio-PRF,Released
5,Jan 16 2023 10:54AM,255091,10,PRF-46717,Bio-PRF,Released
6,Jan 16 2023 10:54AM,255091,10,PRF-46723,Bio-PRF,Released
7,Jan 16 2023 10:54AM,255091,10,PRF-46729,Bio-PRF,Released
8,Jan 16 2023 10:54AM,255091,10,PRF-46734,Bio-PRF,Released
9,Jan 16 2023 10:54AM,255091,10,PRF-46788,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,255087,Released,1
26,255088,Released,1
27,255089,Released,11
28,255091,Released,14
29,255092,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
255087,NaN,1.0
255088,NaN,1.0
255089,NaN,11.0
255091,NaN,14.0
255092,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
255006,40.0,12.0
255049,0.0,42.0
255050,0.0,16.0
255053,0.0,1.0
255054,6.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
255006,40,12
255049,0,42
255050,0,16
255053,0,1
255054,6,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,255006,40,12
1,255049,0,42
2,255050,0,16
3,255053,0,1
4,255054,6,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,255006,40,12
1,255049,,42
2,255050,,16
3,255053,,1
4,255054,6,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC
1,Jan 16 2023 10:54AM,255091,10,Bio-PRF
15,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc."
26,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation
81,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc."
82,Jan 16 2023 10:39AM,255087,21,"NBTY Global, Inc."
83,Jan 16 2023 10:39AM,255086,10,Bio-PRF
84,Jan 16 2023 10:34AM,255084,10,"Senseonics, Incorporated"
86,Jan 16 2023 10:23AM,255080,10,"Citieffe, Inc."
87,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1
1,Jan 16 2023 10:54AM,255091,10,Bio-PRF,,14
2,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc.",,11
3,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation,,55
4,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc.",,1
5,Jan 16 2023 10:39AM,255087,21,"NBTY Global, Inc.",,1
6,Jan 16 2023 10:39AM,255086,10,Bio-PRF,,1
7,Jan 16 2023 10:34AM,255084,10,"Senseonics, Incorporated",,2
8,Jan 16 2023 10:23AM,255080,10,"Citieffe, Inc.",,1
9,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC",,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
1,Jan 16 2023 10:54AM,255091,10,Bio-PRF,14,
2,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc.",11,
3,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation,55,
4,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc.",1,
5,Jan 16 2023 10:39AM,255087,21,"NBTY Global, Inc.",1,
6,Jan 16 2023 10:39AM,255086,10,Bio-PRF,1,
7,Jan 16 2023 10:34AM,255084,10,"Senseonics, Incorporated",2,
8,Jan 16 2023 10:23AM,255080,10,"Citieffe, Inc.",1,
9,Jan 16 2023 10:14AM,255050,15,"Virtus Pharmaceuticals, LLC",16,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1,
1,Jan 16 2023 10:54AM,255091,10,Bio-PRF,14,
2,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc.",11,
3,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation,55,
4,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,NaN
1,Jan 16 2023 10:54AM,255091,10,Bio-PRF,14.0,NaN
2,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc.",11.0,NaN
3,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation,55.0,NaN
4,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 16 2023 10:55AM,255092,22,FUJIFILM Diosynth Biotechnologies Texas LLC,1.0,0.0
1,Jan 16 2023 10:54AM,255091,10,Bio-PRF,14.0,0.0
2,Jan 16 2023 10:45AM,255089,10,"Methapharm, Inc.",11.0,0.0
3,Jan 16 2023 10:42AM,255082,10,ISDIN Corporation,55.0,0.0
4,Jan 16 2023 10:40AM,255088,19,"GUSA Granules USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4081203,260.0,57.0
12,765178,21.0,1.0
15,765105,70.0,40.0
19,255088,1.0,0.0
20,255070,0.0,1.0
21,255087,1.0,0.0
22,510145,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4081203,260.0,57.0
1,12,765178,21.0,1.0
2,15,765105,70.0,40.0
3,19,255088,1.0,0.0
4,20,255070,0.0,1.0
5,21,255087,1.0,0.0
6,22,510145,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,260.0,57.0
1,12,21.0,1.0
2,15,70.0,40.0
3,19,1.0,0.0
4,20,0.0,1.0
5,21,1.0,0.0
6,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,260.0
1,12,Released,21.0
2,15,Released,70.0
3,19,Released,1.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Executing,57.0,1.0,40.0,0.0,1.0,0.0,0.0
Released,260.0,21.0,70.0,1.0,0.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Executing,57.0,1.0,40.0,0.0,1.0,0.0,0.0
1,Released,260.0,21.0,70.0,1.0,0.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Executing,57.0,1.0,40.0,0.0,1.0,0.0,0.0
1,Released,260.0,21.0,70.0,1.0,0.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()